# キャリブレーション


In [1]:
PROGRAM_NAME = "07_calibration_v2"

In [2]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [3]:
payloads = experiment(PROGRAM_NAME)

job_id = 0695e56d-fee2-7246-8000-8949dac53afe
job_status = succeeded  elapsed=750.251


In [4]:
from pprint import pprint

pprint(payloads[0])

{'calib_note': {'hpi_params': {'Q36': {'amplitude': 0.026247334954998588,
                                       'duration': 32.0,
                                       'target': 'Q36',
                                       'tau': 12,
                                       'timestamp': '2026-01-07 13:02:29'},
                               'Q38': {'amplitude': 0.06974395005072237,
                                       'duration': 32.0,
                                       'target': 'Q38',
                                       'tau': 12,
                                       'timestamp': '2026-01-07 13:02:36'},
                               'Q39': {'amplitude': 0.01971344790175394,
                                       'duration': 32.0,
                                       'target': 'Q39',
                                       'tau': 12,
                                       'timestamp': '2026-01-06 11:09:59'},
                               'Q40': {'amplitude': 0.037065484

# calib_note, props, params, classifiersの書き出し

以下のブロックでcalib_note, props, params, classifiersの書き出しを行うので、必要な箇所を書き換えてpushする。  
classifiersはファイルサイズがpushするには大きいようなので、取り扱いに注意！

In [5]:
import json
import os
import yaml


if not os.path.exists("data"):
    os.makedirs("data")

with open(os.path.join("data", "calib_note.json"), "w") as f:
    json.dump(payloads[0].get("calib_note"), f, ensure_ascii=False, indent=2)
with open(os.path.join("data", "props.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("props"), f)
# with open(os.path.join("data", "params.yaml"), "w", encoding="utf-8") as f:
#     yaml.safe_dump(payloads[0].get("params"), f)
with open(os.path.join("data", "classifiers_base64.txt"), "w", encoding="utf-8") as f:
    f.write(payloads[0].get("classifiers"))
